In [13]:
# Colaborador A: Configuración y Preparación de Datos

import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import sagemaker
from sagemaker.experiments import Experiment, Run
import os
import joblib

print("--- Colaborador A Iniciando ---")

# 1. Configurar la Sesión de SageMaker y el Bucket S3
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# *** REEMPLAZA ESTO CON EL NOMBRE DE TU BUCKET S3 REAL ***
bucket = 'mi-bucket-sagemaker-colaboracion-2025'
prefix = 'iris-collab-data' # Carpeta para los datos

print(f"Usando bucket S3: s3://{bucket}/{prefix}")

# 2. Cargar y Preparar el Conjunto de Datos Iris
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.DataFrame(iris.target, columns=['target'])
df = pd.concat([X, y], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Guardar los datos localmente (solo para el script de entrenamiento)
# En un caso real, esto se subiría a S3 y el script lo leería de S3.
# Pero para este ejemplo colaborativo simple, vamos a pasar los datos directamente en memoria
# dentro del experimento para simplificar el flujo del cuaderno.
print("Datos de Iris cargados y divididos.")

# 3. Definir y Entrenar un Modelo (Regresión Logística) dentro de un Experimento
experiment_name = "Iris-Clasificacion-Colaborativa-abc"

# Crear o cargar el Experimento
try:
    experiment = Experiment.load(experiment_name=experiment_name)
    print(f"Reanudando experimento: {experiment_name}")
except Exception:
    experiment = Experiment.create(experiment_name=experiment_name)
    print(f"Creando nuevo experimento: {experiment_name}")

# Iniciar una nueva ejecución para el Colaborador A
with Run(experiment_name=experiment_name, run_name="colaborador-A-logreg") as run:
    run.log_parameters({
        "model_type": "LogisticRegression",
        "random_state": 42,
        "test_size": 0.2,
        "solver": "lbfgs"
    })

    model_logreg = LogisticRegression(max_iter=200, solver='lbfgs')
    model_logreg.fit(X_train, y_train.values.ravel()) # .values.ravel() para evitar UserWarning
    predictions_logreg = model_logreg.predict(X_test)
    accuracy_logreg = accuracy_score(y_test, predictions_logreg)

    run.log_metrics({"accuracy": accuracy_logreg, "colaborador": "A"})
    print(f"Colaborador A: Precisión del modelo de Regresión Logística: {accuracy_logreg:.4f}")

    # Guardar el modelo para este experimento (opcional, pero buena práctica)
    model_path = os.path.join(sagemaker.local.LocalSession().config.output_data_dir, "model_logreg.joblib")
    joblib.dump(model_logreg, model_path)
    run.log_artifact("model_logreg", model_path)
    print(f"Colaborador A: Modelo guardado como artefacto del experimento.")

print("--- Colaborador A Finalizado ---")

--- Colaborador A Iniciando ---
Usando bucket S3: s3://mi-bucket-sagemaker-colaboracion-2025/iris-collab-data
Datos de Iris cargados y divididos.
Creando nuevo experimento: Iris-Clasificacion-Colaborativa-abc


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:64                                                                                   │
│                                                                                                  │
│   61 │   predictions_logreg = model_logreg.predict(X_test)                                       │
│   62 │   accuracy_logreg = accuracy_score(y_test, predictions_logreg)                            │
│   63 │                                                                                           │
│ ❱ 64 │   run.log_metrics({"accuracy": accuracy_logreg, "colaborador": "A"})                      │
│   65 │   print(f"Colaborador A: Precisión del modelo de Regresión Logística: {accuracy_logreg    │
│   66 │                                                                                           │
│   67 │   # Guardar el modelo para este experimento (opcional, pero buena práctica)               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'Run' object has no attribute 'log_metrics'